# **NLP com Flair**
Este tutorial é baseado no livro "Natural Language Processing with Flair".

# 2 - Sequence Tagging

Sequence tagging é a tarefa que envolve "taggear" tokens ou outras unidades de texto. Um exemplo simples
é por exemplo identificar as tokens que são verbos. Um tipo de sequence tagging é o chamado
Named Entity Recognition (NER), que consiste em identificar entidades como organizações, lugares, e pessoas.
Outro exemplo famoso é o que se chama de Part-of-Speech (PoS), que é a tarefa de identificar morfologicamente as tokens (e.g. substantivos, adjetivos, verbos...).

## NER
Vamos iniciar falando um pouco sobre NER.
Infelizmente, o Flair não possui modelos pré-treinados em português. Então vamos usar texto em inglês para
apresentar as funcionalidades.

In [6]:
from flair.data import Sentence
from flair.models import SequenceTagger
tagger = SequenceTagger.load('ner')
sentence = Sentence('Berlin')
tagger.predict(sentence)
print(sentence.to_tagged_string())

/home/davibarreira/miniconda3/envs/nlp/lib/python3.9/site-packages/huggingface_hub/file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-10-18 18:12:58,485 loading file /home/davibarreira/.flair/models/ner-english/4f4cdab26f24cb98b732b389e6cebc646c36f54cfd6e0b7d3b90b25656e4262f.8baa8ae8795f4df80b28e7f7b61d788ecbb057d1dc85aacb316f1bd02837a4a4
2022-10-18 18:13:00,672 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>
Sentence: "Berlin" → ["Berlin"/LOC]


Como o Flair não possui modelos em português para o que queremos, vamos importar um modelo pré-treinado do Hugging Face. 
Esse modelo é um transformer treinando em textos jurídicos em português.

In [14]:
# from transformers import AutoTokenizer, AutoModelForTokenClassification

# # tokenizer = AutoTokenizer.from_pretrained("pierreguillou/ner-bert-large-cased-pt-lenerbr")

# # model = AutoModelForTokenClassification.from_pretrained("pierreguillou/ner-bert-large-cased-pt-lenerbr")
# tagger = SequenceTagger.load("../../ner-bert-large-cased-pt-lenerbr/pytorch_model.bin")

## Part-of-Speech

This has the same interface, we only need to alter our tagger model.

In [16]:
tagger = SequenceTagger.load('pos')
sentence = Sentence('Making a living')
tagger.predict(sentence)
print(sentence.to_tagged_string())

/home/davibarreira/miniconda3/envs/nlp/lib/python3.9/site-packages/huggingface_hub/file_download.py:588: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


2022-10-18 18:32:33,239 loading file /home/davibarreira/.flair/models/pos-english/a9a73f6cd878edce8a0fa518db76f441f1cc49c2525b2b4557af278ec2f0659e.121306ea62993d04cd1978398b68396931a39eb47754c8a06a87f325ea70ac63
2022-10-18 18:32:33,466 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD
Sentence: "Making a living" → ["Making"/VBG, "a"/DT, "living"/NN]


## Avaliando as previsões

Para avaliar o quão preciso nosso tagger é, podemos usar métricas como acurácia (acertos / total). Porém, essa
métrica pode ser ruim caso nosso dataset seja desbalanceado em relação à distribuição das tags. Por exemplo,
se 80% das tags são "x", então um modelo que sempre prevê "x" irá acertar 80% das vezes.

Uma métrica mais interessante é o F1. Essa métrica faz uma média espectral da medida de precisão e revocação (*recall*).
Lembre:

$$
\text{precisao}_i := \frac{tp_i}{tp_i+fp_i}
$$

$$
\text{recall}_i := \frac{tp_i}{tp_i+fn_i}
$$

Considere que temos $n$ tags possíveis. O $tp_i$ é o *true positive* para a tag $i$, i.e. a quantidade de vezes que 
acertamos prevendo a tag $i$. O $fp_i$ é a quantidade de "falso positivo" para tag $i$ e $fn_i$ a quantidade de "falso negativo",
ou seja, uma palavra tinha a tag $i$, mas nosso modelo não classificou com a tag $i$.
Note que a precisão e o recall são calculados para cada tag $i$ separadamente.

O score F1 é então:

$$
F1_i := 2 \cdot \frac{\text{precisao}_i \cdot \text{recall}_i}{\text{precisao}_i + \text{recall}_i}
$$

Temos então uma medida F1 para cada tag do nosso modelo. Se quisermos avaliar o desempenho considerando a previsão
de todas as tags, podemos utilizar tanto o que se chama de *macro* F1, como *micro* F1.

O macro F1 é a média dos F1s, e o micro é a soma dos $tp$, $fp$ e $fn$.
Ou seja, o F1 macro é:

$$
\text{macro F1}:= \frac{\sum_i^n \text{F1}_i}{n}
$$

E para o micro temos:

$$
\text{Pr}:= \frac{\left(\sum_i^n tp_i \right)}{\left(\sum_i^n tp_i \right) + \left(\sum_i^n fp_i \right)}
$$

$$
\text{Re}:= \frac{\left(\sum_i^n tp_i \right)}{\left(\sum_i^n tp_i \right) + \left(\sum_i^n fn_i \right)}
$$

$$
\text{micro F1} := 2 \cdot \frac{Pr \cdot Re}{Pr + Re}
$$